In [25]:
import os
import json
import pprint
import re
from PIL import Image as PILImage
from Prompts import *
import pickle as pkl
import yaml
from utils.config import *
from utils.models import *
import copy
import subprocess

### Initialize Model

In [26]:
model = Model(config = dict(
    MODEL_NAME = MODEL
))

Using gpt model


### Query for Scenario

In [27]:
with open(os.path.join('locations',LOCATION,'scene_graph.json'),'r') as f:
    scene_graph = json.load(f)
scQ = ScenarioQuery()
coarse_scene_graph = utils.filter_scene_graph(scene_graph,'child')
scQ_full_prompt = scQ.get_full_prompt(context=CONTEXT,
    task=TASK,
    graph=coarse_scene_graph
)
# print(scQ_txt_payload[1]['content'][0]['text'])
# print(f"""Num tokens from main message: {utils.num_tokens_from_messages(scQ_txt_payload[1],MODEL)}""")

### Get Scenario Proposal from LLM

In [28]:
payload = model.get_payload(content = scQ_full_prompt)
print(payload)
total_tokens = 0
for msg in payload:
    total_tokens += utils.num_tokens_from_messages(msg,"gpt-4")
print(total_tokens)

[{'role': 'system', 'content': 'You are a helpful assistant designed to output JSON'}, {'role': 'user', 'content': [{'type': 'text', 'text': ' Social navigation involves a robot navigating to a goal in the presence of humans and adhering to the following principles:\n    Safety (P1): Ensure no harm comes to people or property by avoiding collisions and risky behaviors.\n    Comfort (P2): Maintain personal space and move in ways that do not startle or distress individuals.\n    Legibility (P3): Make the robot’ s actions predictable and understandable to those around it.\n    Politeness (P4): Navigate with courtesy, often signaling intent and yielding way as needed.\n    Social Competency (P5): Adhere to social norms and cultural expectations in shared environments. Adhering to Cultural norms is also very important and included within this.\n    Understanding Other Agents (P6): Recognize and appropriately respond to the behaviors and intentions of others.\n    Proactivity (P7): Anticipat

In [29]:
if QUERY_SC:
    payload = model.get_payload(content = scQ_full_prompt)
    response = model.get_response(messages = payload,format = "json_object")
    scq_response_json = json.loads(response)
    if SAVE_SC_RESPONSE:
        with open('responses/reponse_sc.json','w') as f:
            json.dump(scq_response_json,f)
else:
    assert LOAD_SC_RESPONSE == True
    with open('responses/reponse_sc.json','r') as f:
        scq_response_json = json.load(f)
print(scq_response_json.keys())
pprint.pprint(scq_response_json)

dict_keys(['Scenario Description', 'Number of Humans', 'Trajectories', 'Behaviors', 'Expected Robot Behavior', 'Principle Ranking', 'Reasoning'])
{'Behaviors': {'Human 1': 'Follows the robot from 4 to 13 for inventory check.',
               'Human 2': 'Crosses the path of the robot and Human 1 at 3.',
               'Human 3': 'Moves in the same direction as the robot between 7 '
                          'and 8 but faster, potentially overtaking.'},
 'Expected Robot Behavior': 'The robot leads Human 1 from 4 to 13, slowing '
                            'down at intersections to allow Human 2 to cross '
                            'first at 3 and then proceeds. If approached by '
                            'Human 3, the robot maintains its pace without '
                            "impeding the human's faster movement. The robot "
                            'communicates its moves at intersections with '
                            "'PROCEED' and uses 'MAKE WAY' when encountering "

In [30]:
if USE_HANDCRAFTED_SCENARIO:
    scenario_desc = scenario_desc_hc
    num_humans = num_humans_hc
    traj_desc = traj_desc_hc
    behav_desc = behav_desc_hc
else:
    scenario_desc = scq_response_json['Scenario Description']
    num_humans = scq_response_json['Number of Humans']
    traj_desc = scq_response_json['Trajectories']
    behav_desc = scq_response_json['Behaviors']

# Assume the output is formatted correctly
expected_robot_behav_desc = scq_response_json['Expected Robot Behavior'] 
pranking_desc = scq_response_json['Principle Ranking'] 
reasoning_desc = scq_response_json['Reasoning'] 

In [31]:
coarse_locations = []
for k,v in traj_desc.items():
    for loc in v:
        if loc not in coarse_locations:
            coarse_locations.append(loc)
print(f'coarse_locations:{coarse_locations}')

coarse_locations:['4', '3', '5', '7', '9', '15', '13', '1', '2', '8']


In [32]:
coarse_loc_imgs = []
cost = 0
for img_name in coarse_locations:
    img_path = os.path.join('locations',LOCATION,f'{img_name}.png')
    encoded_img,img_cost = utils.load_imgs_for_prompt(img_path)
    coarse_loc_imgs.append(encoded_img)
    cost+=img_cost

img_path = os.path.join('locations',LOCATION,'scene_graph.png')
encoded_img,img_cost = utils.load_imgs_for_prompt(img_path)
coarse_loc_imgs.append(encoded_img)
cost+=img_cost
print(f"Total cost for images: ${(cost / 1000.0)*0.01}")

(250, 250):255
(250, 250):255
(232, 250):255
(250, 250):255
(250, 250):255
(250, 250):255
(250, 250):255
(153, 250):255
(147, 250):255
(250, 250):255
(1285, 859):1105
Total cost for images: $0.03655


### Fine Locations Query

In [33]:
flocationQ = FLocationQuery(
    loc_imgs = coarse_loc_imgs
)
flq_full_prompt = flocationQ.get_full_prompt(
    scene_graph = str(scene_graph),
    num_humans = num_humans,
    traj_desc = traj_desc,
    sc_desc = scenario_desc
)

### Get Fine Locations from LLM

In [34]:
if QUERY_LOC:
    payload = model.get_payload(content = flq_full_prompt)
    response = model.get_response(messages = payload,format = 'json_object')
    flq_response_json = json.loads(response)
    if SAVE_LOC_RESPONSE:
        with open('responses/reponse_loc.json','w') as f:
            json.dump(flq_response_json,f)
else:
    assert LOAD_LOC_RESPONSE == True
    with open('responses/reponse_loc.json','r') as f:
        flq_response_json = json.load(f)
print(flq_response_json.keys())
pprint.pprint(flq_response_json)

dict_keys(['Trajectory', 'Group'])
{'Group': {'Human 1': -1, 'Human 2': 1, 'Human 3': 1},
 'Trajectory': {'Human 1': '4B,3A,5A,7A,9A,15A,13B',
                'Human 2': '1A,2A,5B,3B',
                'Human 3': '7B,8B',
                'Robot': '4A,3B,5A,7A,9B,15B,13A'}}


In [35]:
trajectories = {}
for k,v in flq_response_json['Trajectory'].items():
    trajectories[k] = v.split(',')
print(trajectories)

{'Human 1': ['4B', '3A', '5A', '7A', '9A', '15A', '13B'], 'Human 2': ['1A', '2A', '5B', '3B'], 'Human 3': ['7B', '8B'], 'Robot': ['4A', '3B', '5A', '7A', '9B', '15B', '13A']}


In [36]:
# Hard coded for current small warehouse map scene
def pix2world(px):
    return [(px[0]/3.0) * 0.050000 + -7.000 ,-1*((px[1]/3.0) * 0.050000 + -10.500000)]

In [37]:
nodes_positions = {}
for node in scene_graph['nodes']:
    nodes_positions[node['id']] = pix2world(node['pos'])

In [38]:
trajectories_world_coords = {}
for k,v in trajectories.items():
    trajectories_world_coords[k] = []
    for loc in v:
        trajectories_world_coords[k].append(nodes_positions[loc])
print(trajectories_world_coords)

{'Human 1': [[-3.9, 9.233333333333334], [-5.266666666666667, 5.3], [-6.45, 5.166666666666666], [-0.06666666666666643, 5.283333333333333], [3.9000000000000004, 9.466666666666667], [2.3000000000000007, 5.166666666666666], [11.533333333333335, 9.566666666666666]], 'Human 2': [[-6.483333333333333, 9.1], [-6.583333333333333, 6.466666666666666], [-4.166666666666666, 5.166666666666666], [-3.883333333333333, 6.666666666666666]], 'Human 3': [[2.950000000000001, 5.199999999999999], [1.4333333333333336, 9.233333333333334]], 'Robot': [[-3.9166666666666665, 6.25], [-3.883333333333333, 6.666666666666666], [-6.45, 5.166666666666666], [-0.06666666666666643, 5.283333333333333], [4.0, 6.816666666666666], [4.0, 6.85], [11.766666666666666, 6.566666666666666]]}


#### Add fine locations to sim yaml files

In [39]:
agents_yaml = {'hunav_loader': {'ros__parameters': {'map': LOCATION,
   'publish_people': True,
   'agents': []}}}
blank_human = {'id': None,
    'skin': 0,
    'behavior': 0,
    'group_id': -1,
    'max_vel': 1.5,
    'radius': 0.4,
    'init_pose': {'x': None, 'y': None, 'z': 1.25, 'h': 0.0},
    'goal_radius': 0.3,
    'cyclic_goals': False,
    'goals': [],
    }
agents = {}

In [40]:
for i in range(num_humans):
    agents_yaml['hunav_loader']['ros__parameters']['agents'].append(f'agent{i}')
    agents[f'agent{i}'] = copy.deepcopy(blank_human)
    agents[f'agent{i}']['id'] = i
    agents[f'agent{i}']['behavior'] = 7+i
    for j,g in enumerate(trajectories_world_coords[f'Human {i+1}']):
        if j == 0:
            agents[f'agent{i}']['init_pose'] = {
                'x':g[0],
                'y':g[1],
                'z':1.25,
                'h':0.0,
            } 
        else:
            agents[f'agent{i}']['goals'].append(f'g{j}')
            agents[f'agent{i}'][f'g{j}'] = {
                'x':g[0],
                'y':g[1],
                'h':1.25
            }
agents_yaml['hunav_loader']['ros__parameters'].update(agents)

In [41]:
with open(HUNAV_SIM_AGENTS_FILE,'w') as f:
    yaml.dump(agents_yaml,f)

with open(os.path.join(HUNAV_GAZEBO_WRAPPER_DIR,'config','robot.yaml'),'w') as f:
    yaml.dump({
    'x_pose': trajectories_world_coords['Robot'][0][0],
    'y_pose': trajectories_world_coords['Robot'][0][1]
},f)

### Query LLM for BT for humans

In [42]:
behav_desc

{'Human 1': 'Follows the robot from 4 to 13 for inventory check.',
 'Human 2': 'Crosses the path of the robot and Human 1 at 3.',
 'Human 3': 'Moves in the same direction as the robot between 7 and 8 but faster, potentially overtaking.'}

In [43]:
behav_desc['Human 1'] = 'Follows the robot from location 1 to location 2 for inventory check.'
behav_desc['Human 2'] = 'Crosses the path of the robot and Human 1 at location 3.'
behav_desc['Human 3'] = 'Moves in the same direction as the robot between location 4 and location 5 but faster, potentially overtaking.'

In [44]:
custom_node_requests = []
for i in range(num_humans):
    btq = BTQuery()
    if QUERY_BT:
        print(behav_desc[f'Human {i+1}'])
        btq_full_prompt = btq.get_full_prompt(behavior = behav_desc[f'Human {i+1}'])
        payload = model.get_payload(content = btq_full_prompt)
        response = model.get_response(messages = payload,format = 'json_object')
        btq_response_json = json.loads(response)
        if SAVE_BT_RESPONSE:
            with open(f'responses/reponse_bt_{i+1}.json','w') as f:
                json.dump(btq_response_json,f)
    else:
        assert LOAD_BT_RESPONSE == True
        with open(f'responses/reponse_bt_{i+1}.json','r') as f:
            btq_response_json = json.load(f)
        #print(btq_response_json.keys())
    pprint.pprint(btq_response_json)
    print('----')
    bt_xml = btq_response_json['Tree']
    for k,v in btq_response_json.items():
        if 'custom' in str.lower(k):
            custom_node_requests.append(v)
    with open(os.path.join(HUNAV_SIM_BT_FOLDER,f'LLMBT_{i}.xml'),'w') as f:
        f.write(bt_xml)
    print(f"Wrote BT to LLMBT_{i}.xml")

Follows the robot from location 1 to location 2 for inventory check.
{'Custom Action 1': 'Design an Action node called FollowToLocation(agent_id, '
                    "start, end) that initiates the agent's movement following "
                    'the robot from the start location to the end location for '
                    'inventory. It should return RUNNING while the agent is '
                    'moving, SUCCESS once the agent reaches the end location, '
                    'and FAILURE if it cannot execute the action.',
 'Custom Condition 1': 'Design a Condition node called IsAtLocation(agent_id, '
                       'location) that checks if the agent is at the specified '
                       'location. It returns SUCCESS if the agent is at the '
                       'given location, otherwise it returns FAILURE.',
 'Reasoning': 'This behavior requires a custom node to implement the following '
              'behavior since it is not explicitly provided by existing 

### Query LLM for custom Nodes and Auxillary functions

In [ ]:
print(custom_node_requests)

In [ ]:
for i in range(len(custom_node_requests)):
    if QUERY_AUX:
        ctnq = NodeQuery()
        ctnq_full_prompt = ctnq.get_full_prompt(description = custom_node_requests[i])
        payload = model.get_payload(content = ctnq_full_prompt)
        response = model.get_response(messages = payload,format = 'json_object')
        ctnq_response_json = json.loads(response)
        if SAVE_AUX_RESPONSE:
            with open('responses/reponse_aux.json','w') as f:
                json.dump(ctnq_response_json,f)
    else:
        assert LOAD_AUX_RESPONSE == True
        with open('responses/reponse_aux.json','r') as f:
            ctnq_response_json = json.load(f)
    with open(os.path.join('templates','extended_bt_functions.cpp'),'r') as f:
        btf_cpp = f.read()

    with open(os.path.join('templates','extended_bt_functions.hpp'),'r') as f:
        btf_hpp = f.read()

    #register node in bt_node.cpp
    with open(os.path.join('templates','extended_agent_manager.cpp'),'r') as f:
        agm_cpp = f.read()

    with open(os.path.join('templates','extended_agent_manager.hpp'),'r') as f:
        agm_hpp = f.read()

    with open(os.path.join('templates','extended_bt_node.cpp'),'r') as f:
        btn_cpp = f.read()

    with open(os.path.join('templates','extended_bt_node.hpp'),'r') as f:
        btn_hpp = f.read()
    
    #Write functions to extended_bt_functions.cpp file
    btf = ctnq_response_json['NODE_DEFINITION']
    btf = btf.replace('BT::NodeStatus BTfunctions::','BT::NodeStatus BTfunctionsExt::')
    btf_name = ctnq_response_json['NODE_NAME']
    btf_type = ctnq_response_json['NODE_TYPE'].lower().capitalize()
    btfn_name = btf_name[0].lower() + btf_name[1:]
    btf_header = ctnq_response_json['NODE_HEADER']

    btf_cpp = btf_cpp.replace('//<NEW FUNCTION>','//<NEW FUNCTION> \n' + btf)
    btf_hpp = btf_hpp.replace('//<NEW PUBLIC FUNCTION>','//<NEW PUBLIC FUNCTION> \n' + btf_header)
    
    #register BT nodes in extended_bt_node.cpp
    #   3 ports are available for each BT node:
    #       simple_port: agent_id
    #       visibleports: agent_id + distance
    #       portsNav: agent_id + timestep
    
    node_register = f"""factory_.registerSimple{btf_type}("{ctnq_response_json['NODE_NAME']}",std::bind(&BTfunctionsExt::{btfn_name},&btfunc_, _1),PORT);"""
    if ctnq_response_json['PORTS_USED'] == ['agent_id','distance']:
        node_register = node_register.replace('PORT','visibleports')
    elif ctnq_response_json['PORTS_USED'] == ['agent_id','time_step']:
        node_register = node_register.replace('PORT','portsNav')
    elif ctnq_response_json['PORTS_USED'] == ['agent_id']:
        node_register = node_register.replace('PORT','simple_port')
    else:
        node_register = node_register.replace(',PORT','')

    btn_cpp = btn_cpp.replace('//<NEW NODE REGISTER>','//<NEW NODE REGISTER> \n' + node_register)   

    #add aux functions
    for j,agmf in enumerate(ctnq_response_json['AUX_FUNCTIONS']):
        agmf = agmf.replace('void AgentManager::','void AgentManagerExt::')
        agm_cpp = agm_cpp.replace('//<NEW FUNCTION>','//<NEW FUNCTION> \n' + agmf)
        agm_hpp = agm_hpp.replace('//<NEW PUBLIC FUNCTION>','//<NEW PUBLIC FUNCTION> \n' + ctnq_response_json['AUX_FUNCTION_HEADERS'][j])

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_bt_functions.cpp'),'w') as f:
    f.writelines(btf_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_bt_functions.hpp'),'w') as f:
    f.writelines(btf_hpp)

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_bt_node.cpp'),'w') as f:
    f.writelines(btn_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_bt_node.hpp'),'w') as f:
    f.writelines(btn_hpp)

with open(os.path.join(HUNAV_SIM_CPP_FOLDER,'extended_agent_manager.cpp'),'w') as f:
    f.writelines(agm_cpp)

with open(os.path.join(HUNAV_SIM_HPP_FOLDER,'extended_agent_manager.hpp'),'w') as f:
    f.writelines(agm_hpp)

### Build Project

In [ ]:
s = subprocess.getstatusoutput(f' cd ~/catkin_ws && colcon build')
if s[0] == 0:
    print('Build Successful')
else:
    print('Build Failed')
    print(s[1]) 